# Display spatial changes of NDVI density clusters

Author: Morgane Magnier (morgane.magnier@vattenfall.com)

Copyright © 2024 Magnier Morgane 

This notebook is part of a thesis project. The copyright of the thesis itself belongs to the student Morgane Magnier.  

**Rights and Intellectual Property**:  
- Vattenfall has the right to use the findings, methods, and conclusions of this thesis in its operations.  
- Any material generated within the framework of this thesis that is subject to intellectual property protection (e.g., source code, computer program, design, or invention) belongs to Vattenfall, unless otherwise agreed in writing.  

Permission is granted to view, copy, and share this notebook for **educational or personal purposes only**, provided that this notice is included in all copies.  

---

In [ ]:
import ee, eemont, geemap
import numpy as np
import geemap.colormaps as cm
import matplotlib.pyplot as plt
import changes_btw_2_dates
import kmeans_clustering
import sys
sys.path.append('../water_detection')
import wetlands_unsupervised_clustering

ee.Authenticate()
ee.Initialize()

## Preprocessing

In [2]:
def get_s2_sr_cld_col(roi, start_date, end_date):
    # Import and filter S2 SR.
    CLOUD_FILTER = 80 
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(roi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.calendarRange(5,9, 'month'))
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(roi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.calendarRange(4,10, 'month')))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    CLD_PRB_THRESH = 40

    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):

    NIR_DRK_THRESH = 0.15
    CLD_PRJ_DIST = 2

    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    BUFFER = 100
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

def count_pixels(image,roi): 
    pixel_count = image.select('B1').reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=roi,
        scale=10,
        maxPixels=1e9
    ).get('B1')
    return image.set('pixel_count', pixel_count)


def get_clouds_free_collection_in_roi(roi,start_date, end_date,CLOUD_FILTER = 80,POURCENTAGE_MAX_CLOUDS = 80):
    
    s2_sr_cld_col = get_s2_sr_cld_col(roi, start_date, end_date)

    #s2_col_masked = s2_col.map(mask_s2_clouds).map(lambda image : image.clip(roi))

    s2_sr_masked = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                .map(apply_cld_shdw_mask)).map(lambda image : image.clip(roi))

    # Calculate the threshold for pixel count (80% of the reference image)

    nb_pixels_time_series = s2_sr_masked.map(lambda image: count_pixels(image, roi))

    # Get the image with the maximum pixel count
    max_pixel_count_image = nb_pixels_time_series.sort('pixel_count', False).first()
    ref_img_pixel_count = max_pixel_count_image.get('pixel_count').getInfo()
    pixel_count_threshold = ref_img_pixel_count * 0.95

    # Filter the collection based on the pixel count threshold
    filtered_s2_sr_masked = nb_pixels_time_series.filter(ee.Filter.gte('pixel_count', pixel_count_threshold))

    return filtered_s2_sr_masked

### Collections

In [3]:
start_date = ee.Date('2017-07-06')
end_date = ee.Date('2023-07-10')

roi = ee.Geometry.Polygon([[[17.204933,60.402663],[17.204933,60.455525],[17.2645,60.455525],[17.2645,60.402663],[17.204933,60.402663]]])
img_1 = get_clouds_free_collection_in_roi(roi,start_date, start_date.advance(1,'day')).spectralIndices(['NDVI','NDWI','BI', 'NDGI','SeLI']).first()
img_2 = get_clouds_free_collection_in_roi(roi,end_date, end_date.advance(1,'day')).spectralIndices(['NDVI','NDWI','BI', 'NDGI','SeLI']).first()

In [4]:
# Visualize
import geemap
m = geemap.Map()
rgbVis = {'bands': ["B4", "B3", "B2"], 'min': 0, 'max': 0.1 }
m.addLayer(img_1.scale().clip(roi), rgbVis, 'img')
m.addLayer(wetlands_contours.updateMask(wetlands_contours),{'min': 0, 'max': 1, 'palette': ['blue']})
m.centerObject(roi,14)
m

NameError: name 'wetlands_contours' is not defined

In [ ]:
display(img_1)
display(img_2)

### Wetlands

In [ ]:
max_water_date = ee.Date('2018-05-09')
min_water_date = ee.Date('2022-09-25')
wetlands = wetlands_unsupervised_clustering.getWetlandsS2(roi, min_water_date, max_water_date)
wetlands_contours = wetlands.subtract(wetlands.focalMin(20, 'square', 'meters'))

In [ ]:
display(wetlands)

In [ ]:
m = geemap.Map()
m.centerObject(roi,14)
rgbVis = {'bands': ['B4','B3', 'B2'], 'min' : 0 , 'max' : 0.1}
m.addLayer(img_2.scale().clip(roi),rgbVis,'test')
m.addLayer(wetlands.updateMask(wetlands),{'min': 0, 'max': 1, 'palette': ['blue']})
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
m = geemap.Map()
m.centerObject(roi,14)
ndgiVis = {
  'min': 0.4,
  'max': 1,
  'palette' : [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301'],
}
ndwiVis = {
    'min': -1,
    'max': 1,
    'palette' : [
        '#FFFFFF', '#E5F5FC', '#CCE6F4', '#B2DFF0', '#99D8EB', '#80D1E6',
        '#66CAE2', '#4DC3DD', '#33BCD9', '#1AB5D4', '#00AED0', '#00A7CB',
        '#009FC6', '#0098C2', '#0091BD', '#008AB8', '#0083B4', '#007CAF',
        '#0075AA', '#006EA6', '#0067A1', '#005F9D', '#005898', '#005193',
        '#004A8F', '#00438A', '#003C85', '#003481', '#002D7C', '#002677',
        '#001F72', '#00186D', '#001169', '#000A64', '#00035F', '#00005B'
    ]
}
bsiVis = {
    'min': -1,
    'max': 1,
    'palette' : [
        '#FF4500',  # Orange-rouge pour les valeurs négatives
        '#FFA500',  # Orange pour les valeurs proches de zéro
        '#FFD700'   # Or pour les valeurs positives
    ]
}
m.addLayer(img_1.select('BI').clip(roi),bsiVis,'lai')
m.addLayer(wetlands_contours.updateMask(wetlands_contours),{'min': 0, 'max': 1, 'palette': ['black']})
m.add_colorbar(bsiVis, label="BI")
#m

## Kmeans clustering

In [ ]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
nClusters = 4

clusters_img_1 = kmeans_clustering.kmeans_clustering_S2(img_1.resample('bicubic'), roi, bands = bands, nClusters = nClusters)
clusters_img_2 = kmeans_clustering.kmeans_clustering_S2(img_2.resample('bicubic'), roi, bands = bands, nClusters = nClusters)

In [ ]:
display(clusters_img_1)
display(clusters_img_2)

In [6]:
# Visualize
nClusters=4
import geemap
m = geemap.Map()
# Display the RGB cluster means.
rgbVis = {'bands': ["B4", "B3", "B2"], 'min': 0, 'max': 0.1 }
kmeans_vis = {'min': 0, 'max': nClusters - 1, 'palette': [ '#1E90FF', '#BDB76B', '#BC8F8F', '#FF4500', '#228B22']}

m.centerObject(roi,14)
m.addLayer(clusters_img_2.visualize(**kmeans_vis).clip(roi), {}, 'Unsupervised K-means Classification', True, 1)
m.addLayer(wetlands_contours.updateMask(wetlands_contours),{'min': 0, 'max': 1, 'palette': ['blue']})
m

NameError: name 'clusters_img_2' is not defined

### Identification for each clusters

In [ ]:
clusters_annotated_img_1 = kmeans_clustering.identify_clusters(img_1, roi, clusters_img_1, nClusters = 4)
clusters_annotated_img_2 = kmeans_clustering.identify_clusters(img_2, roi, clusters_img_2, nClusters = 4)

In [ ]:
display(clusters_annotated_img_2)
display(clusters_annotated_img_1)

In [ ]:
kmeans_clustering.plot_clusters_in_wetlands_S2(img_1, roi, clusters_annotated_img_1, wetlands)

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

### Mask clusters

In [ ]:
water_1, grass_1, trees_1, bare_land_1 = kmeans_clustering.cut_img_by_clusters(img_1, clusters_annotated_img_1)
water_2, grass_2, trees_2, bare_land_2 = kmeans_clustering.cut_img_by_clusters(img_2, clusters_annotated_img_2)

In [ ]:
m = geemap.Map()
m.centerObject(roi,14)
rgbVisS2 = {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max':  0.1}
m.addLayer(water_1.scale(), rgbVisS2, 'test')
m

In [ ]:
m = geemap.Map()
m.centerObject(roi,14)
rgbVisS2 = {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max':  0.1}
m.addLayer(bare_land_1.scale(), rgbVisS2, 'test')
m

In [ ]:
m = geemap.Map()
m.centerObject(roi,14)
rgbVisS2 = {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max':  0.1}
m.addLayer(grass_1.selfMask().scale(), rgbVisS2, 'test')
m

EEException: Image.select: Band pattern '1_trees' did not match any bands. Available bands: [cluster]

In [ ]:
m = geemap.Map()
m.centerObject(roi,14)
rgbVisS2 = {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max':  0.1}
m.addLayer(trees_1.scale(), rgbVisS2, 'test')
m

## Changes detection

### In bare land

In [ ]:
# Comparer les deux images pixel par pixel

grass_1 = clusters_annotated_img_1.eq(2)
grass_2 = clusters_annotated_img_2.eq(2)

difference_masked = (
    img_1.select('B1')
    .where(grass_1, 1)
    .where(grass_2, -1)
    .where(grass_1.Not().And(grass_2.Not()), 0)
    .where(grass_1.And(grass_2), 0)
)

display(difference_masked)
mask = difference_masked.neq(0)
difference_masked = difference_masked.updateMask(mask)

In [ ]:
changeVis = {
    'palette': ['fa1373','2659eb'],
    'min': -1,
    'max': 1
}
mb = geemap.Map()
#mb.addLayer(composite, {'bands': ['B4',  'B3',  'B2'], 'min': 0.0, 'max': 0.3}, 'True color')
mb.addLayer(img_1.clip(roi).scale(), {'min': 0, 'max': 0.1, 'bands': ['B4',  'B3',  'B2']}, 'img')
mb.addLayer(difference_masked.updateMask(wetlands), changeVis, 'changes')
mb.centerObject(roi, 14)
mb.addLayer(wetlands_contours.updateMask(wetlands_contours),{'min': 0, 'max': 1, 'palette': ['blue']})
mb

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

### In trees

In [ ]:
# Comparer les deux images pixel par pixel

bare_1 = clusters_annotated_img_1.eq(3)
bare_2 = clusters_annotated_img_2.eq(3)

difference_masked = (
    img_1.select('B1')
    .where(bare_1, 1)
    .where(bare_2, -1)
    .where(bare_1.Not().And(bare_2.Not()), 0)
    .where(bare_1.And(bare_2), 0)
)

display(difference_masked)
mask = difference_masked.neq(0)
difference_masked = difference_masked.updateMask(mask)

changeVis = {
    'palette': ['fa1373','2659eb'],
    'min': -1,
    'max': 1
}
mb = geemap.Map()
#mb.addLayer(composite, {'bands': ['B4',  'B3',  'B2'], 'min': 0.0, 'max': 0.3}, 'True color')
roi = ee.Geometry.Polygon([[[17.246647,60.418937],[17.201843,60.418937],[17.201843,60.444815],[17.246647,60.444815],[17.246647,60.418937]]])
mb.addLayer(img_1.clip(roi).scale(), {'min': 0, 'max': 0.1, 'bands': ['B4',  'B3',  'B2']}, 'img')
mb.addLayer(difference_masked.updateMask(wetlands), changeVis, 'changes')
mb.centerObject(roi, 14)
mb.addLayer(wetlands_contours.updateMask(wetlands_contours),{'min': 0, 'max': 1, 'palette': ['blue']})
mb

Map(center=[60.431876162888365, 17.224244999997214], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
m = geemap.Map()
m.centerObject(roi,14)
m.addLayer(img_2.scale().clip(roi), rgbVis, 'img_1')
m.addLayer(diff_trees_land.updateMask(wetlands).updateMask(trees_1),diff_vis_params)
m.addLayer(diff_trees_land.updateMask(wetlands).updateMask(trees_2),diff_vis_params)
m

Map(center=[60.431876162888365, 17.224244999997214], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Sélectionner la bande '1_grass' pour chaque image
grass_1 = clusters_bands_img_1.select('2_grass')
grass_2 = clusters_bands_img_2.select('2_grass')

# Créer une image masquée indiquant les différences entre les deux images pour la classe '1_grass'
diff_grass = grass_2.neq(grass_1)
diff_grass = ee.Image(1).where(diff_grass, 0)

# Définir les palettes de couleurs pour visualiser les différences
palette_diff = ['#800080', 'red']  
palette_diff_2 = ['#800080', 'blue']  

# Paramètres de visualisation pour les images de différences
diff_vis_params = {'min': 0, 'max': 1, 'palette': palette_diff}
diff_vis_params_2 = {'min': 0, 'max': 1, 'palette': palette_diff_2}

# Créer une image des contours des différences entre grass_1 et grass_2
diff_grass_contours = diff_grass.subtract(diff_grass.focalMin(20, 'square', 'meters'))

# Afficher l'image de différences grass_1 neq grass_2
display(diff_grass, diff_vis_params)

{'min': 0, 'max': 1, 'palette': ['#800080', 'red']}

In [ ]:
# Créer une carte avec geemap
m = geemap.Map()
m.centerObject(roi, 14)
# Ajouter les images sur la carte
m.addLayer(img_2.scale().clip(roi), rgbVis, 'img_1')  # Visualiser img_1 avec la palette rgbVis
# Ajouter les couches de différences sur la carte avec masquage
m.addLayer(diff_grass.updateMask(wetlands).updateMask(grass_2), diff_vis_params_2, 'Différences grass_2 neq grass_1')
m.addLayer(diff_grass.updateMask(wetlands).updateMask(grass_1), diff_vis_params, 'Différences grass_1 neq grass_2')

# Afficher la carte
m

Map(center=[60.431876162888365, 17.224244999997214], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Créer une carte avec geemap
m = geemap.Map()
m.centerObject(roi, 14)
# Ajouter les images sur la carte
m.addLayer(img_2.scale().clip(roi), rgbVis, 'img_1')  # Visualiser img_1 avec la palette rgbVis
# Ajouter les couches de différences sur la carte avec masquage
m.addLayer(trees_1.updateMask(wetlands).selfMask(),{'palette': ['blue']}, 'Différences grass_2 neq grass_1')
m

Map(center=[60.431876162888365, 17.224244999997214], controls=(WidgetControl(options=['position', 'transparent…